In [8]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NUMTRADES", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "TOTALOFFERQTY", "WEIGHTEDAVGOFFERPX", "TOTALBIDQTY", "WEIGHTEDAVGBIDPX"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "NUMORDERS_B1", "NOORDERS _B1", "ORDERQTY_B1", "NUMORDERS_S1", "NOORDERS _S1", "ORDERQTY_S1"]
columns5 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]


year = "2016"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[162:]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
    
    # SZ version A without AM PM, 分卷
    if len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***.7z.001')[0]).split('.')[0]
        os.system("copy /b " + n_ame + ".7z.* " + n_ame + ".7z")
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\***'))) == 1:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        for pp in np.array(glob.glob(path1 +'\\***')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot = pd.concat([snapshot, s1])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A without AM PM")
        
    # SZ version A AM PM
    # I:\\LEVEL2_shenzhen\\2016\\04\\19\\SZL2_SNAPSHOT_20160419_AM.7z 58
    # I:\\LEVEL2_shenzhen\\2016\\05\\06\\SZL2_SNAPSHOT_20160506_AM.txt 59
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))) == 1:
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, txt")
                
        
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0].split('_')[2]
        os.mkdir(path + '\\' + d_ate)
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        n_ame1 = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame1 + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame1 + '.7z')
            bad.append(data + "\\" + n_ame1 + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"] 
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot = snapshot[snapshot["time"] != 0]
        
        print("SZ version A AM PM, .7z")
 
    # am & pm
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\09\\am_snap_level_spot.7z'
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\10\\am_snap_level_spot.7z.001'
    
    elif len(np.array(glob.glob(data +'\\am_hq_snap_spot.7z'))) == 1:
        date = glob.glob(data +'\\am_hq_snap_spot.7z')[0][19:23] + glob.glob(data +'\\am_hq_snap_spot.7z')[0][24:26] + glob.glob(data +'\\am_hq_snap_spot.7z')[0][27:29]
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + date)
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5     
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        date = glob.glob(data +'\\am_hq_snap_spot.7z.001')[0][19:23] + glob.glob(data +'\\am_hq_snap_spot.7z.001')[0][24:26] + glob.glob(data +'\\am_hq_snap_spot.7z.001')[0][27:29]
        path = r'F:\SZ\2016' 
        os.mkdir(path + '\\' + date)
        os.chdir(data)
        os.system("copy /b am_hq_snap_spot.7z.* am_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_hq_snap_spot.7z')
            bad.append(data + '\\am_hq_snap_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_hq_snap_spot.7z.* pm_hq_snap_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_hq_snap_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_hq_snap_spot.7z')
            bad.append(data + '\\pm_hq_snap_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
      
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = (snapshot['OrigTime'] - int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷")
   
  

    snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
    snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)
    s2 = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID").first().reset_index()
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: str(x//1000000000))
    s2["OrigTime"] = s2["OrigTime"].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
    s2 = s2.rename(columns={"StockID": "ID", "OrigTime":"date", 'openPrice':"d_open", "PreClosePx":"d_yclose","HighPx":"d_high", "LowPx":"d_low", "close":"d_close", "cum_volume":"d_volume", "cum_amount":"d_amount"})
    s2 = s2[["ID", "date", "d_open", "d_yclose", "d_high", "d_low", "d_close", "d_volume", "d_amount"]]
    
    # 1. check massive missing   
    date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
    date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
    date["group"] = date["time"]//30000
    snapshot["group"] = snapshot["time"]//30000
    gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
    if len(set(gl) - set(snapshot["group"].unique())) == 0:
        s2["massive_missing"] = 0
    else:
        s2["massive_missing"] = 1
 
    # 2. check each security missing
    snapshot = snapshot.sort_values(by=["StockID", "OrigTime"])
    dd = snapshot[snapshot["cum_volume"] == snapshot["cum_max"]].groupby("StockID")["time"].first().reset_index()
    
    data2 = snapshot[(snapshot["time"] > 92500000) & (snapshot["time"] <= 145700000) & (snapshot["cum_volume"] > 0)][["StockID", "cum_volume",
                                                                                                                     "OrigTime", "time"]]
    del snapshot
    data2 = data2.sort_values(by=["StockID", "OrigTime"])    
    data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
    data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
    data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
    data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
    data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
    data2 = data2.rename(columns={"75%": "75%_time"})
    vu = data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].apply(lambda x: x.describe([0.9])["90%"]).reset_index()
    vu = vu.rename(columns={"volume_update":"90%"})
    data2 = pd.merge(data2, vu, on="StockID")
    data2["90%"] = data2["90%"].apply(lambda x: max(1000, x))
    data2["indicator"] = 0
    data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
    data2 = data2[data2["indicator"] != 1]    
    if data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["90%"]) & (data2["75%_time"] <= 4)].shape[0] == 0:
        s2["security_missing"] = 0
    else:
        s2["security_missing"] = 1
        
    # 3. check close auction
    dd["auction"] = np.where(dd["time"]<=145700000, 0, 1)
    dd = dd.rename(columns={"StockID": "ID"})
    s2 = pd.merge(s2, dd[["ID", "auction"]], on="ID")
    
    s2["ID"] = s2["ID"].astype(str)
    s2["ID"] = s2["ID"].apply(lambda x: "SZ" + x.rjust(6, "0"))
    df += [s2]
    print(s2.head())
    print(data + " finished~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
df = pd.concat(df).reset_index(drop=True)
df.to_pickle(r"F:\\SZ2016_4.pkl")
print(bad)

20160829000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-08-29    9.41      9.45    9.45   9.38     9.42  56523188   
1  SZ000002  2016-08-29   22.86     22.89   23.25  22.75    22.91  78244619   
2  SZ000004  2016-08-29    0.00     36.83    0.00   0.00     0.00         0   
3  SZ000005  2016-08-29    7.39      7.39    7.63   7.37     7.51  19773810   
4  SZ000006  2016-08-29    8.28      8.26    8.35   8.20     8.29  11386146   

       d_amount  massive_missing  security_missing  auction  
0  5.322954e+08                0                 0        1  
1  1.796321e+09                0                 0        1  
2  0.000000e+00                0                 0        0  
3  1.486532e+08                0                 0        1  
4  9.432696e+07                0                 0        1  
J:\LEVEL2_shenzhen\2016\08\29 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20160830000000000
am & pm data 分卷
         ID 

20160909000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-09-09    9.40      9.40    9.43   9.36     9.38  32743143   
1  SZ000002  2016-09-09   23.50     23.62   23.59  23.01    23.17  88429677   
2  SZ000004  2016-09-09    0.00     36.83    0.00   0.00     0.00         0   
3  SZ000005  2016-09-09    7.62      7.59    7.62   7.47     7.47  13663902   
4  SZ000006  2016-09-09    8.23      8.23    8.24   8.10     8.11  11162162   

       d_amount  massive_missing  security_missing  auction  
0  3.075273e+08                0                 0        1  
1  2.050212e+09                0                 0        1  
2  0.000000e+00                0                 0        0  
3  1.030620e+08                0                 0        1  
4  9.101458e+07                0                 0        1  
J:\LEVEL2_shenzhen\2016\09\09 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20160910000000000
am & pm data 分卷
         ID 

20160923000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2016-09-23    9.16      9.16    9.19   9.14     9.15   28879877   
1  SZ000002  2016-09-23   26.32     27.00   26.55  25.24    25.48  197838372   
2  SZ000004  2016-09-23   38.51     36.83   40.51  37.82    40.51   13500207   
3  SZ000005  2016-09-23    7.53      7.52    7.57   7.38     7.40    9902551   
4  SZ000006  2016-09-23    8.95      9.02    9.15   8.88     9.11   40853836   

       d_amount  massive_missing  security_missing  auction  
0  2.646995e+08                0                 0        1  
1  5.106198e+09                0                 0        1  
2  5.288980e+08                0                 0        1  
3  7.390124e+07                0                 0        1  
4  3.685414e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\09\23 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20160924000000000
am & pm data 分卷
      

20161012000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2016-10-12    9.14      9.15    9.16   9.12     9.13   27031620   
1  SZ000002  2016-10-12   26.00     26.23   26.67  25.63    26.60   87775400   
2  SZ000004  2016-10-12   40.40     40.40   42.30  39.63    40.37    5653416   
3  SZ000005  2016-10-12    7.46      7.50    7.50   7.40     7.50    9684912   
4  SZ000006  2016-10-12   10.20     10.43   10.95  10.11    10.70  104699570   

       d_amount  massive_missing  security_missing  auction  
0  2.469955e+08                0                 0        1  
1  2.296507e+09                0                 0        1  
2  2.303692e+08                0                 0        1  
3  7.206411e+07                0                 0        1  
4  1.102598e+09                0                 0        1  
J:\LEVEL2_shenzhen\2016\10\12 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161013000000000
am & pm data 分卷
      

20161025000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-10-25    9.25      9.24    9.26   9.18     9.21  50075292   
1  SZ000002  2016-10-25   25.20     25.24   25.34  24.80    25.01  71149209   
2  SZ000004  2016-10-25   39.62     39.83   39.84  39.14    39.72   1968877   
3  SZ000005  2016-10-25    7.52      7.51    7.55   7.48     7.53  10432761   
4  SZ000006  2016-10-25   10.06     10.09   10.30  10.00    10.21  34063570   

       d_amount  massive_missing  security_missing  auction  
0  4.615427e+08                0                 0        1  
1  1.779391e+09                0                 0        1  
2  7.771362e+07                0                 0        1  
3  7.836978e+07                0                 0        1  
4  3.470156e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\10\25 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161026000000000
am & pm data 分卷
         ID 

20161107000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-11-07    9.10      9.11    9.12   9.07     9.12  49724012   
1  SZ000002  2016-11-07   24.47     24.59   24.58  24.03    24.16  41989356   
2  SZ000004  2016-11-07   47.40     46.97   47.40  44.70    45.33   4458262   
3  SZ000005  2016-11-07    7.38      7.40    7.58   7.35     7.53  22869718   
4  SZ000006  2016-11-07   10.23     10.31   10.53  10.06    10.10  28757447   

       d_amount  massive_missing  security_missing  auction  
0  4.524479e+08                0                 0        1  
1  1.016202e+09                0                 0        1  
2  2.031176e+08                0                 0        1  
3  1.713990e+08                0                 0        1  
4  2.932801e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\11\07 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161108000000000
am & pm data 分卷
         ID 

20161118000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close   d_volume  \
0  SZ000001  2016-11-18    9.21      9.21    9.21   9.16     9.18   51759734   
1  SZ000002  2016-11-18   27.80     27.00   29.00  27.51    27.72  150316488   
2  SZ000004  2016-11-18   46.55     46.88   47.30  46.00    46.12    2090168   
3  SZ000005  2016-11-18    7.55      7.55    7.60   7.48     7.56   14375279   
4  SZ000006  2016-11-18   10.04     10.00   10.04   9.82     9.82   22139604   

       d_amount  massive_missing  security_missing  auction  
0  4.752900e+08                0                 0        1  
1  4.252381e+09                0                 0        1  
2  9.713458e+07                0                 0        1  
3  1.084103e+08                0                 0        1  
4  2.198565e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\11\18 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161121000000000
am & pm data 分卷
      

20161201000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-12-01    9.57      9.55    9.63   9.55     9.60  64600437   
1  SZ000002  2016-12-01   26.56     26.98   27.30  26.30    26.97  95017059   
2  SZ000004  2016-12-01   41.85     42.18   42.44  41.37    41.82   1021033   
3  SZ000005  2016-12-01    7.44      7.42    7.48   7.41     7.45   8795164   
4  SZ000006  2016-12-01    9.56      9.60    9.71   9.54     9.67  16055496   

       d_amount  massive_missing  security_missing  auction  
0  6.196337e+08                0                 0        1  
1  2.546520e+09                0                 0        1  
2  4.278643e+07                0                 0        1  
3  6.544822e+07                0                 0        1  
4  1.544686e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\12\01 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161202000000000
am & pm data 分卷
         ID 

20161214000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-12-14    9.42      9.42    9.51   9.40     9.40  59770574   
1  SZ000002  2016-12-14   22.67     23.06   22.90  22.20    22.29  41281457   
2  SZ000004  2016-12-14   40.48     40.48   42.00  40.20    41.20   1275748   
3  SZ000005  2016-12-14    6.83      6.85    6.89   6.72     6.76   6381301   
4  SZ000006  2016-12-14    8.79      8.90    9.50   8.66     9.12  29898754   

       d_amount  massive_missing  security_missing  auction  
0  5.647875e+08                0                 0        1  
1  9.315383e+08                0                 0        1  
2  5.235186e+07                0                 0        1  
3  4.338016e+07                0                 0        1  
4  2.715258e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\12\14 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161215000000000
am & pm data 分卷
         ID 

20161227000000000
am & pm data 分卷
         ID        date  d_open  d_yclose  d_high  d_low  d_close  d_volume  \
0  SZ000001  2016-12-27    9.12      9.12    9.13   9.07     9.08  26884124   
1  SZ000002  2016-12-27   20.65     20.65   21.98  20.59    21.42  74330548   
2  SZ000004  2016-12-27   43.57     43.74   45.20  43.57    44.58   1309062   
3  SZ000005  2016-12-27    6.87      6.87    6.90   6.84     6.87   4145329   
4  SZ000006  2016-12-27    9.57      9.61    9.80   9.50     9.63  30777771   

       d_amount  massive_missing  security_missing  auction  
0  2.447883e+08                0                 0        1  
1  1.584455e+09                0                 0        1  
2  5.855171e+07                0                 0        1  
3  2.847926e+07                0                 0        1  
4  2.972326e+08                0                 0        1  
J:\LEVEL2_shenzhen\2016\12\27 finished~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
20161228000000000
am & pm data 分卷
         ID 

In [29]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","NUMORDERS_B1","NOORDERS_B1","ORDERQTY_B1",
                    "NUMORDERS_S1","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NUMTRADES", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "TOTALOFFERQTY", "WEIGHTEDAVGOFFERPX", "TOTALBIDQTY", "WEIGHTEDAVGBIDPX"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "NUMORDERS_B1", "NOORDERS _B1", "ORDERQTY_B1", "NUMORDERS_S1", "NOORDERS _S1", "ORDERQTY_S1"]
columns5 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "wa_offerPrice", "totalbidqty", "wa_bidPrice", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]

snap1 = pd.read_table("F:\\SZ\\2016\\20160411\\SZL2_SNAPSHOT_20160411_AM.txt", header=None)
assert(snap1.shape[1] == len(columns3))
snap1.columns = columns3
snap2 = pd.read_table("F:\\SZ\\2016\\20160411\\SZL2_SNAPSHOT_20160411_PM.txt", header=None)
assert(snap2.shape[1] == len(columns3))
snap2.columns = columns3
snapshot = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
del snap1
del snap2
snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
snapshot['time'] = snapshot["DATATIMESTAMP"] 
snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
snapshot = snapshot[snapshot["time"] != 0]

snapshot = snapshot[(snapshot["StockID"] < 4000) | (snapshot["StockID"] > 300000)]
snapshot["cum_max"] = snapshot.groupby("StockID")["cum_volume"].transform(max)


date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
date["group"] = date["time"]//30000
snapshot["group"] = snapshot["time"]//30000
gl = date[((date["time"] >= 93000000) & (date["time"] < 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
set(gl) - set(snapshot["group"].unique())


{3478,
 3480,
 3481,
 3483,
 3484,
 3485,
 3486,
 3487,
 3488,
 3490,
 3491,
 3493,
 3494,
 3495,
 3496,
 3497,
 3498,
 3500,
 3501,
 3503,
 3504,
 3505,
 3506,
 3507,
 3508,
 3510,
 3511,
 3513,
 3514,
 3515,
 3516,
 3517,
 3518,
 3520,
 3521,
 3523,
 3524,
 3525,
 3526,
 3527,
 3528,
 3530,
 3531,
 3666,
 3667,
 3668,
 3670,
 3671,
 3673,
 3674,
 3675,
 3676,
 3677,
 3678,
 3680,
 3681,
 3683,
 3684,
 3685,
 3686,
 3687,
 3688,
 3690,
 3691,
 3693,
 3694,
 3695,
 3696,
 3697,
 3698,
 3700,
 3701,
 3703,
 3704,
 3705,
 3706,
 3707,
 3708,
 3710,
 3711,
 3713,
 3714,
 3715,
 3716,
 3717,
 3718,
 3720,
 3721,
 3723,
 3724,
 3725,
 3726,
 3727,
 3728,
 3730,
 3731,
 3733,
 3734,
 3735,
 3736,
 3737,
 3738,
 3740,
 3741,
 3743,
 3744,
 3745,
 3746,
 3747,
 3748,
 3750,
 3751,
 3753,
 3754,
 3755,
 3756,
 3757,
 3758,
 3760,
 3761,
 4813,
 4814,
 4815,
 4816,
 4817,
 4818,
 4820,
 4821,
 4823,
 4824,
 4825,
 4826,
 4827,
 4828,
 4830,
 4831,
 4833,
 4834,
 4835,
 4836,
 4837,
 4838,
 4840,

In [30]:
snapshot.groupby("StockID")["time"].max().max()

144331460

In [27]:
date[date["group"] == 4843]

,Orig,time,group
22980,2019-06-10 14:53:00,145300000,4843
22981,2019-06-10 14:53:01,145301000,4843
22982,2019-06-10 14:53:02,145302000,4843
22983,2019-06-10 14:53:03,145303000,4843
22984,2019-06-10 14:53:04,145304000,4843
22985,2019-06-10 14:53:05,145305000,4843
22986,2019-06-10 14:53:06,145306000,4843
22987,2019-06-10 14:53:07,145307000,4843
22988,2019-06-10 14:53:08,145308000,4843
22989,2019-06-10 14:53:09,145309000,4843


In [3]:
# df = pd.concat(df).reset_index(drop=True)
# df.to_pickle(r"F:\\SZ2016_2.pkl")
# print(bad)
df

,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount,massive_missing,security_missing,auction
0,SZ000001,2016-05-05,10.69,10.72,10.72,10.65,10.72,21886499,2.338828e+08,1,0,0
1,SZ000005,2016-05-05,7.33,7.34,7.39,7.28,7.33,8752340,6.413026e+07,1,0,0
2,SZ000006,2016-05-05,8.09,8.09,8.11,8.03,8.08,8997714,7.262225e+07,1,0,0
3,SZ000008,2016-05-05,11.00,10.94,11.04,10.87,10.98,11044437,1.209186e+08,1,0,0
4,SZ000009,2016-05-05,13.92,13.96,14.12,13.88,14.04,34906223,4.883042e+08,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
141158,SZ300532,2016-08-26,41.64,37.85,41.64,41.64,41.64,4000,1.665600e+05,0,0,0
141159,SZ300533,2016-08-26,94.45,85.86,94.45,94.45,94.45,4800,4.533600e+05,0,0,0
141160,SZ300535,2016-08-26,69.09,62.81,69.09,69.09,69.09,11447,7.908732e+05,0,0,0
141161,SZ300538,2016-08-26,20.92,15.85,22.82,20.92,22.82,8036,1.824315e+05,0,0,0
